<h1 style="color: skyblue; text-align: center;">Big Data Analytics</h1>









<h3>1. Collecte de Données :</h3>
           <p> On va utiliser le Dataset suivant</p>



 [Significant Earthquakes, 1965-2016](https://www.kaggle.com/datasets/usgs/earthquake-database/data)

Puis on va copier le dataset dans HDFS sous le repertoire input 

<h3>2.Préparation des Données :</h3>


In [1]:
!pip install pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession  
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("etl") \
        .master("spark://hadoop-master:7077") \
        .config("spark.jars.packages", "net.snowflake:snowflake-jdbc:3.13.14,net.snowflake:spark-snowflake_2.12:2.10.0-spark_3.0") \
        .config("spark.executor.memory", "2g") \
        .config("spark.driver.memory", "8g") \
        .getOrCreate()


print("Spark Version:", spark.version)

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
net.snowflake#snowflake-jdbc added as a dependency
net.snowflake#spark-snowflake_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-83ca0419-1e6b-4ca5-a3cc-9914b1b19d90;1.0
	confs: [default]
	found net.snowflake#snowflake-jdbc;3.13.14 in central
	found net.snowflake#spark-snowflake_2.12;2.10.0-spark_3.0 in central
	found net.snowflake#snowflake-ingest-sdk;0.10.3 in central
:: resolution report :: resolve 361ms :: artifacts dl 10ms
	:: modules in use:
	net.snowflake#snowflake-ingest-sdk;0.10.3 from central in [default]
	net.snowflake#snowflake-jdbc;3.13.14 from central in [default]
	net.snowflake#spark-snowflake_2.12;2.10.0-spark_3.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicte

Spark Version: 3.2.4


In [4]:
# Charger le dataset 
df_load = spark.read.csv("hdfs:///user/root/input/database.csv", header=True, inferSchema=True)
# Aperçu de df_load
df_load.take(1)

[Row(Date='01/02/1965', Time='13:44:18', Latitude=19.246, Longitude=145.616, Type='Earthquake', Depth=131.6, Depth Error=None, Depth Seismic Stations=None, Magnitude=6.0, Magnitude Type='MW', Magnitude Error=None, Magnitude Seismic Stations=None, Azimuthal Gap=None, Horizontal Distance=None, Horizontal Error=None, Root Mean Square=None, ID='ISCGEM860706', Source='ISCGEM', Location Source='ISCGEM', Magnitude Source='ISCGEM', Status='Automatic')]

In [4]:
# Supprimer les champs dont nous n'avons pas besoin de df_load.
lst_dropped_columns = ['Depth Error', 'Time', 'Depth Seismic Stations','Magnitude Error','Magnitude Seismic Stations','Azimuthal Gap',
                       'Horizontal Distance','Horizontal Error',
    'Root Mean Square','Source','Location Source','Magnitude Source','Status']

df_load = df_load.drop(*lst_dropped_columns)
# Aperçu df_load
df_load.show(5)

[Stage 3:>                                                          (0 + 1) / 1]

+----------+--------+---------+----------+-----+---------+--------------+------------+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|
+----------+--------+---------+----------+-----+---------+--------------+------------+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|      6.0|            MW|ISCGEM860706|
|01/04/1965|   1.863|  127.352|Earthquake| 80.0|      5.8|            MW|ISCGEM860737|
|01/05/1965| -20.579| -173.972|Earthquake| 20.0|      6.2|            MW|ISCGEM860762|
|01/08/1965| -59.076|  -23.557|Earthquake| 15.0|      5.8|            MW|ISCGEM860856|
|01/09/1965|  11.938|  126.427|Earthquake| 15.0|      5.8|            MW|ISCGEM860890|
+----------+--------+---------+----------+-----+---------+--------------+------------+
only showing top 5 rows



In [5]:
# Créer un champ année et l'ajouter au dataframe.
df_load = df_load.withColumn('Year', year(to_timestamp('Date', 'dd/MM/yyyy')))
# Aperçu df_load
df_load.show(5)

+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|Year|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|      6.0|            MW|ISCGEM860706|1965|
|01/04/1965|   1.863|  127.352|Earthquake| 80.0|      5.8|            MW|ISCGEM860737|1965|
|01/05/1965| -20.579| -173.972|Earthquake| 20.0|      6.2|            MW|ISCGEM860762|1965|
|01/08/1965| -59.076|  -23.557|Earthquake| 15.0|      5.8|            MW|ISCGEM860856|1965|
|01/09/1965|  11.938|  126.427|Earthquake| 15.0|      5.8|            MW|ISCGEM860890|1965|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
only showing top 5 rows



In [6]:
# Construire le dataframe de fréquence en utilisant le champ année et les comptages pour chaque année.
df_quake_freq = df_load.groupBy('Year').count().withColumnRenamed('count', 'Counts')
# Aperçu df_quake_freq
df_quake_freq.show(5)

[Stage 7:>                                                          (0 + 1) / 1]

+----+------+
|Year|Counts|
+----+------+
|1990|   196|
|1975|   150|
|1977|   148|
|2003|   187|
|2007|   211|
+----+------+
only showing top 5 rows



In [7]:
# Aperçu df_load schema
df_load.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Depth: double (nullable = true)
 |-- Magnitude: double (nullable = true)
 |-- Magnitude Type: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)



In [8]:
# Convertir certains champs de chaîne de caractères en types numériques.
df_load = df_load.withColumn('Latitude', df_load['Latitude'].cast(DoubleType()))\
    .withColumn('Longitude', df_load['Longitude'].cast(DoubleType()))\
    .withColumn('Depth', df_load['Depth'].cast(DoubleType()))\
    .withColumn('Magnitude', df_load['Magnitude'].cast(DoubleType()))

# Aperçu df_load
df_load.show(5)

+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|Year|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|      6.0|            MW|ISCGEM860706|1965|
|01/04/1965|   1.863|  127.352|Earthquake| 80.0|      5.8|            MW|ISCGEM860737|1965|
|01/05/1965| -20.579| -173.972|Earthquake| 20.0|      6.2|            MW|ISCGEM860762|1965|
|01/08/1965| -59.076|  -23.557|Earthquake| 15.0|      5.8|            MW|ISCGEM860856|1965|
|01/09/1965|  11.938|  126.427|Earthquake| 15.0|      5.8|            MW|ISCGEM860890|1965|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
only showing top 5 rows



In [9]:
# Aperçu df_load schema
df_load.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Depth: double (nullable = true)
 |-- Magnitude: double (nullable = true)
 |-- Magnitude Type: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)



In [10]:
# Créer les champs de magnitude moyenne (avg magnitude) et de magnitude maximale (max magnitude) et les ajouter à df_quake_freq.
df_max = df_load.groupBy('Year').max('Magnitude').withColumnRenamed('max(Magnitude)', 'Max_Magnitude')
df_avg = df_load.groupBy('Year').avg('Magnitude').withColumnRenamed('avg(Magnitude)', 'Avg_Magnitude')

In [11]:
# Joindre df_max et df_avg à df_quake_freq.
df_quake_freq = df_quake_freq.join(df_avg, ['Year']).join(df_max, ['Year'])
# Aperçu df_quake_freq
df_quake_freq.show(5)

[Stage 17:>                                                         (0 + 1) / 1]

+----+------+-----------------+-------------+
|Year|Counts|    Avg_Magnitude|Max_Magnitude|
+----+------+-----------------+-------------+
|1990|   196|5.858163265306125|          7.6|
|1975|   150| 5.84866666666667|          7.8|
|1977|   148|5.757432432432437|          7.6|
|2003|   187|5.850802139037435|          7.6|
|2007|   211| 5.89099526066351|          8.4|
+----+------+-----------------+-------------+
only showing top 5 rows



In [12]:
# Supprimer les valeurs nulles.
df_load.dropna()
df_quake_freq.dropna()

DataFrame[Year: int, Counts: bigint, Avg_Magnitude: double, Max_Magnitude: double]

In [13]:
# Aperçu dataframes
df_load.show(5)

[Stage 18:>                                                         (0 + 1) / 1]

+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|Year|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|      6.0|            MW|ISCGEM860706|1965|
|01/04/1965|   1.863|  127.352|Earthquake| 80.0|      5.8|            MW|ISCGEM860737|1965|
|01/05/1965| -20.579| -173.972|Earthquake| 20.0|      6.2|            MW|ISCGEM860762|1965|
|01/08/1965| -59.076|  -23.557|Earthquake| 15.0|      5.8|            MW|ISCGEM860856|1965|
|01/09/1965|  11.938|  126.427|Earthquake| 15.0|      5.8|            MW|ISCGEM860890|1965|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
only showing top 5 rows



In [14]:
df_quake_freq.show(5)

+----+------+-----------------+-------------+
|Year|Counts|    Avg_Magnitude|Max_Magnitude|
+----+------+-----------------+-------------+
|1990|   196|5.858163265306125|          7.6|
|1975|   150| 5.84866666666667|          7.8|
|1977|   148|5.757432432432437|          7.6|
|2003|   187|5.850802139037435|          7.6|
|2007|   211| 5.89099526066351|          8.4|
+----+------+-----------------+-------------+
only showing top 5 rows



<h3>3.Stockage des Données :</h3>


In [16]:
# Ecrire df_load dans HDFS.

df_load = df_load.withColumnRenamed("Magnitude Type", "Magnitude_Type")

df_load.write.mode("overwrite").parquet("hdfs:///user/root/Quake/quakes")

In [17]:
# Ecrire df_quake_freq dans HDFS


df_quake_freq.write.mode("overwrite").parquet("hdfs:///user/root/Quake/quake_freq")

<h3>4.Modélisation et Analyse :</h3>


In [19]:
# Charger le fichier de données de test dans un dataframe.
df_test = spark.read.csv("hdfs:///user/root/input/database.csv", header=True)
# Aperçu df_test
df_test.take(1)

[Row(Date='01/02/1965', Time='13:44:18', Latitude='19.246', Longitude='145.616', Type='Earthquake', Depth='131.6', Depth Error=None, Depth Seismic Stations=None, Magnitude='6', Magnitude Type='MW', Magnitude Error=None, Magnitude Seismic Stations=None, Azimuthal Gap=None, Horizontal Distance=None, Horizontal Error=None, Root Mean Square=None, ID='ISCGEM860706', Source='ISCGEM', Location Source='ISCGEM', Magnitude Source='ISCGEM', Status='Automatic')]

In [20]:
df_train = spark.read.parquet("hdfs:///user/root/Quake/quakes/")
# Aperçu df_train
df_train.show(5)

[Stage 41:>                                                         (0 + 1) / 1]

+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude_Type|          ID|Year|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|      6.0|            MW|ISCGEM860706|1965|
|01/04/1965|   1.863|  127.352|Earthquake| 80.0|      5.8|            MW|ISCGEM860737|1965|
|01/05/1965| -20.579| -173.972|Earthquake| 20.0|      6.2|            MW|ISCGEM860762|1965|
|01/08/1965| -59.076|  -23.557|Earthquake| 15.0|      5.8|            MW|ISCGEM860856|1965|
|01/09/1965|  11.938|  126.427|Earthquake| 15.0|      5.8|            MW|ISCGEM860890|1965|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
only showing top 5 rows



In [21]:
# Sélectionner les champs que nous utiliserons et supprimer les champs dont nous n'avons pas besoin.
df_test_clean = df_test['time', 'latitude', 'longitude', 'Magnitude', 'depth']
# Aperçu  df_test_clean
df_test_clean.show(5)

+--------+--------+---------+---------+-----+
|    time|latitude|longitude|Magnitude|depth|
+--------+--------+---------+---------+-----+
|13:44:18|  19.246|  145.616|        6|131.6|
|11:29:49|   1.863|  127.352|      5.8|   80|
|18:05:58| -20.579| -173.972|      6.2|   20|
|18:49:43| -59.076|  -23.557|      5.8|   15|
|13:32:50|  11.938|  126.427|      5.8|   15|
+--------+--------+---------+---------+-----+
only showing top 5 rows



In [22]:
# Renommer les champs.
df_test_clean = df_test_clean.withColumnRenamed('time', 'Date')\
    .withColumnRenamed('latitude', 'Latitude')\
    .withColumnRenamed('longitude', 'Longitude')\
    .withColumnRenamed('depth', 'Depth')

# Aperçu df_test_clean
df_test_clean.show(5)

[Stage 43:>                                                         (0 + 1) / 1]

+--------+--------+---------+---------+-----+
|    Date|Latitude|Longitude|Magnitude|Depth|
+--------+--------+---------+---------+-----+
|13:44:18|  19.246|  145.616|        6|131.6|
|11:29:49|   1.863|  127.352|      5.8|   80|
|18:05:58| -20.579| -173.972|      6.2|   20|
|18:49:43| -59.076|  -23.557|      5.8|   15|
|13:32:50|  11.938|  126.427|      5.8|   15|
+--------+--------+---------+---------+-----+
only showing top 5 rows



In [23]:
# Aperçu Schema
df_test_clean.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Magnitude: string (nullable = true)
 |-- Depth: string (nullable = true)



In [24]:
# Convertir certains champs de chaîne de caractères en champs numériques.
df_test_clean = df_test_clean.withColumn('Latitude', df_test_clean['Latitude'].cast(DoubleType()))\
    .withColumn('Longitude', df_test_clean['Longitude'].cast(DoubleType()))\
    .withColumn('Depth', df_test_clean['Depth'].cast(DoubleType()))\
    .withColumn('Magnitude', df_test_clean['Magnitude'].cast(DoubleType()))

In [25]:
df_test_clean.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Magnitude: double (nullable = true)
 |-- Depth: double (nullable = true)



In [26]:
# Créer les dataframes d'entraînement et de test.
df_testing = df_test_clean['Latitude', 'Longitude', 'Magnitude', 'Depth']
df_training = df_train['Latitude', 'Longitude', 'Magnitude', 'Depth']

In [27]:
# Aperçu df_training
df_training.show(5)

[Stage 44:>                                                         (0 + 1) / 1]

+--------+---------+---------+-----+
|Latitude|Longitude|Magnitude|Depth|
+--------+---------+---------+-----+
|  19.246|  145.616|      6.0|131.6|
|   1.863|  127.352|      5.8| 80.0|
| -20.579| -173.972|      6.2| 20.0|
| -59.076|  -23.557|      5.8| 15.0|
|  11.938|  126.427|      5.8| 15.0|
+--------+---------+---------+-----+
only showing top 5 rows



In [28]:
# Aperçu df_testing
df_testing.show(5)

+--------+---------+---------+-----+
|Latitude|Longitude|Magnitude|Depth|
+--------+---------+---------+-----+
|  19.246|  145.616|      6.0|131.6|
|   1.863|  127.352|      5.8| 80.0|
| -20.579| -173.972|      6.2| 20.0|
| -59.076|  -23.557|      5.8| 15.0|
|  11.938|  126.427|      5.8| 15.0|
+--------+---------+---------+-----+
only showing top 5 rows



In [29]:
#  Supprimer les enregistrements avec des valeurs nulles de nos dataframes.
df_testing = df_testing.dropna()
df_training = df_training.dropna()

In [31]:
!pip install numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 22.4 kB/s eta 0:00:0000:01:00:04


In [32]:
import numpy as np
print(np.__version__)


1.24.4


In [33]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [35]:
# Sélectionner les caractéristiques à analyser dans notre modèle, puis créer le vecteur de caractéristiques.
assembler = VectorAssembler(inputCols=['Latitude', 'Longitude', 'Depth'], outputCol='features')

# Créer le modèle
model_reg = RandomForestRegressor(featuresCol='features', labelCol='Magnitude')

# Chaîner l'assembleur avec le modèle dans une pipeline.
pipeline = Pipeline(stages=[assembler, model_reg])

# Entraîner le modèle.
model = pipeline.fit(df_training)

# Faire la prédiction.
pred_results = model.transform(df_testing)

In [36]:
# Aperçu pred_results dataframe
pred_results.show(5)

[Stage 60:>                                                         (0 + 1) / 1]

+--------+---------+---------+-----+--------------------+-----------------+
|Latitude|Longitude|Magnitude|Depth|            features|       prediction|
+--------+---------+---------+-----+--------------------+-----------------+
|  19.246|  145.616|      6.0|131.6|[19.246,145.616,1...|5.856788614374566|
|   1.863|  127.352|      5.8| 80.0|[1.863,127.352,80.0]|5.848285859448586|
| -20.579| -173.972|      6.2| 20.0|[-20.579,-173.972...|5.946778356205064|
| -59.076|  -23.557|      5.8| 15.0|[-59.076,-23.557,...|5.975950606279386|
|  11.938|  126.427|      5.8| 15.0|[11.938,126.427,1...|5.960209013105624|
+--------+---------+---------+-----+--------------------+-----------------+
only showing top 5 rows



In [37]:
# Évaluer le modèle
# rmse doit être inférieur à 0,5 pour que le modèle soit utile
evaluator = RegressionEvaluator(labelCol='Magnitude', predictionCol='prediction', metricName='rmse')
rmse = evaluator.evaluate(pred_results)
print('Root Mean Squared Error (RMSE) on test data = %g' % rmse)

[Stage 61:>                                                         (0 + 1) / 1]

Root Mean Squared Error (RMSE) on test data = 0.415346


In [2]:
"""
Créer le dataset de prédiction.
"""

'\nCréer le dataset de prédiction.\n'

In [39]:
# Créer le dataset de prédiction.
df_pred_results = pred_results['Latitude', 'Longitude', 'prediction']

# Renommer le champ de prédiction.
df_pred_results = df_pred_results.withColumnRenamed('prediction', 'Pred_Magnitude')

# Ajouter plus de colonnes
df_pred_results = df_pred_results.withColumn('Year', lit(2025))\
    .withColumn('RMSE', lit(rmse))

# Aperçu df_pred_results
df_pred_results.show(5)

[Stage 62:>                                                         (0 + 1) / 1]

+--------+---------+-----------------+----+-------------------+
|Latitude|Longitude|   Pred_Magnitude|Year|               RMSE|
+--------+---------+-----------------+----+-------------------+
|  19.246|  145.616|5.856788614374566|2025|0.41534627540254415|
|   1.863|  127.352|5.848285859448586|2025|0.41534627540254415|
| -20.579| -173.972|5.946778356205064|2025|0.41534627540254415|
| -59.076|  -23.557|5.975950606279386|2025|0.41534627540254415|
|  11.938|  126.427|5.960209013105624|2025|0.41534627540254415|
+--------+---------+-----------------+----+-------------------+
only showing top 5 rows



In [40]:
# Charger le dataset de prediction dans hdfs
df_pred_results.write.mode("overwrite").parquet("hdfs:///user/root/Quake/pred_results")



<h3>5.Visualisation des Résultats :</h3>

1️⃣ Installer le connecteur Snowflake pour Spark pour transferer la BD de hdfs a snowflake


In [5]:

sfOptions = {
    "sfURL" : "https://vv85366.switzerland-north.azure.snowflakecomputing.com",
    "sfDatabase" : "Quake",
    "sfSchema" : "PUBLIC",
    "sfWarehouse" : "COMPUTE_WH",
    "sfRole" : "ACCOUNTADMIN",  
    "sfUser" : "ADMIN",
    "sfPassword" : "BGKUzh72xKF4aCA"
}


In [6]:
# Charger la table Quake depuis HDFS
quake_df_1 = spark.read.parquet("hdfs:///user/root/Quake/quakes")
quake_df_2 = spark.read.parquet("hdfs:///user/root/Quake/quake_freq")
quake_df_3 = spark.read.parquet("hdfs:///user/root/Quake/pred_results")


In [8]:
# Transférer la première table a Snowflake 
quake_df_1.write \
    .format("snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "quakes") \
    .mode("overwrite") \
    .save()

# Transférer la deuxième table a Snowflake
quake_df_2.write \
    .format("snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "quake_freq") \
    .mode("overwrite") \
    .save()

# Transférer la troisième table a snowflake
quake_df_3.write \
    .format("snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "pred_results") \
    .mode("overwrite") \
    .save()

2025-02-14 15:44:59,083 WARN snowflake.SnowflakeConnectorUtils$: Query pushdown is not supported because you are using Spark 3.2.4 with a connector designed to support Spark 3.0. Either use the version of Spark supported by the connector or install a version of the connector that supports your version of Spark.
2025-02-14 15:46:45,788 WARN snowflake.SnowflakeConnectorUtils$: Query pushdown is not supported because you are using Spark 3.2.4 with a connector designed to support Spark 3.0. Either use the version of Spark supported by the connector or install a version of the connector that supports your version of Spark.
2025-02-14 15:46:47,155 WARN snowflake.SnowflakeConnectorUtils$: Query pushdown is not supported because you are using Spark 3.2.4 with a connector designed to support Spark 3.0. Either use the version of Spark supported by the connector or install a version of the connector that supports your version of Spark.
2025-02-14 15:48:08,961 WARN snowflake.SnowflakeConnectorUtil

Enfin, nous analyserons les données en créant un tableau de bord avec Tableau software


[Lien vers Tableau ](https://public.tableau.com/app/profile/hind.cherrat/viz/Classeur2_17342840465670/BigData?publish=yes)
